In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

Kütüphaneleri import ettiğimiz blok
Numpy: array işlemleri ve sayısal hesaplamalar için kullanılır.
TensorFlow: Derin öğrenme modelleri geliştirmek ve yapay nöronlar oluşturmak için kullanılır.
Dense: Yapay sinir ağı oluşturmada kullanılır.
Dropout: regularization sınıf için kullanılır.
MinMaxScaler: Veriyi 0 , 1 aralığında ölçeklemek veya normalleştirmek için kullanılır


In [ ]:
np.random.seed(42)
tf.random.set_seed(42)

In [6]:
import cv2
cv2.imshow("Image", image)
cv2.waitKey(0)
cv2.destroyAllWindows()

Rasgele sayılar oluşturuyoruz.
Buradaki 42 nin özel bir anlamı yoktur. Popüler olduğu için kullanılır.
Bu değerler ağın figure 1 de görünen ilk ağırlıklarını oluşturacaktır. 


In [ ]:

timesteps = 500
time = np.linspace(0, 1, timesteps)


Veri setinde kullanmak için zaman vektörü oluşturuyoruz

In [ ]:
SoC = np.sin(2 * np.pi * time) * 50 + 50 + np.random.normal(0, 2, timesteps)

Veri setindeki şarj durumunu simule edebilmek için ürettiğimiz fonksiyon. Şarj durumu şarj olurken arada şarjı azaltmaktadır ve gerçekçi veri elde etmek için gürültü ekliyoruz.

In [ ]:

SoH = 100 - 0.05 * time * timesteps + np.random.normal(0, 0.5, timesteps)


Zaman ilerledikçe batarya sağlının o.05 azalacak şekilde bir fonksiyonda azalmasını sağlıyoruz. Gerçeği simule edebilmek için gürültü ekliyoruz 

In [ ]:
scaler = MinMaxScaler(feature_range=(0, 1))
data_scaled = scaler.fit_transform(np.column_stack((SoC, SoH)))

scaler = MinMaxScaler(feature_range=(0, 1)): Veriler 0-1 aralığına ölçeklenecek.

np.column_stack((SoC, SoH)): SoC ve SoH tek bir 2 boyutlu diziye (shape: (500, 2)) birleştiriliyor.

scaler.fit_transform(...): Veriyi öğren ve dönüştür (her sütunu ayrı ayrı 0-1 aralığına çeker).

In [ ]:
seq_length = 5  
X, y = [], []
for i in range(len(data_scaled) - seq_length):
    X.append(data_scaled[i:i + seq_length])  
    y.append(data_scaled[i + seq_length])  
    
X, y = np.array(X), np.array(y)

Modelimize geçmiş 5 örneği verip 6. örneği tahmin etmeye çalışıyoruz

X.append(data_scaled[i:i + seq_length]): i’den i+5’e kadar olan 5 satırlık (SoC, SoH) parçayı X’e ekle.

y.append(data_scaled[i + seq_length]): Bir sonraki adımın SoC, SoH değerini y’ye ekle. (Yani X->5 zaman adımı, y->1 zaman adımı)

X ve Y değerlerini model girişi için array'e çeviriyoruz

In [ ]:
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

Verinin %80'i eğitim %20'si test verisi olarak kullanıyoruz

In [ ]:
X_train = X_train.reshape(X_train.shape[0], -1)
X_test = X_test.reshape(X_test.shape[0], -1)

X_train.shape[0]: örnek sayısı, -1 demek geriye kalan boyutu otomatik bul.
---------------------------------------------
X_Train orjinal biçimi şu şekildedir.
(num_samples, seq_length, num_features)

num_samples: Kaç tane eğitim örneğiniz (sample) var? (ör. 395)

seq_length: Kaç zaman adımı alıyorsunuz? (seq_length = 5)

num_features: Her zaman adımında kaç değişken var? (SoC, SoH => 2)

Dolayısıyla X_train’in boyutu (num_samples, 5, 2) gibi olabilir.

Ancak FFNN için tipik olarak 2 boyutlu girdi beklemektedir 
------------------------------------------
Örnek girdi (num_samples, input_dimension)

num_samples: Örnek sayısı (satır sayısı)

input_dimension: Her örnekteki özellik (feature) sayısı (sütun sayısı)

Elimizde 5 zaman adımı ve 2 değişken olduğu için, bunu “tek bir vektör” (yani 1 boyutlu bir satır) gibi ele almak istiyoruz. Toplam 10 sayı (5 * 2 = 10) olduğuna göre, her örnek 10 boyutlu bir vektör şeklinde FFNN’e girsin.
---------------------------------------------
X_train.reshape(X_train.shape[0], -1)
X_train.shape[0]: kaç örnek (sample) olduğunu belirtir (ör. 395).

-1 demek, “geri kalanı otomatik hesapla” anlamına gelir. Python/NumPy, 5*2 = 10 boyutunu bilir, dolayısıyla her örneği (395, 10) boyutuna getirir.

Bu sayede X_train (num_samples, 5, 2) şeklinden (num_samples, 10) şekline dönmüş olur. Yani her örnek, 10 sütuna sahip düz (flatten) bir vektöre çevrilir. Bu FFNN’in anlayabileceği formattır, çünkü FFNN “gizli katman” girişlerinde sabit boyutlu bir vektör bekler.


![alt text][def]

[def]: <../Desktop/AI/Driver Drowsiness Detection 2/image.png>

In [ ]:

model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dropout(0.2),
    Dense(2) 
])

1) model = Sequential([...])
Sequential: Keras API’si içinde katmanları sıralı (düz bir şerit halinde) eklediğimiz basit bir model tanımlama yöntemidir.

Katmanlar, girilen sıraya göre veriyi işler. Her katmandan çıkan sonuç, bir sonraki katmana girdi olur.
-----------------------------------------------------------
2) Dense(64, activation='relu', input_shape=(10,))
Dense(64): Bu katmanda 64 nöron (yapay nöron) bulunur.

Tam bağlı (fully-connected) bir katmandır; önceki katmanın (ya da giriş katmanının) her bir çıktısı, bu katmandaki her nörona bağlanır.

activation='relu': ReLU (Rectified Linear Unit) aktivasyon fonksiyonu kullanılır. ReLU, girdi değeri negatifse 0, pozitifse kendisi olacak şekilde çıktı üretir.

Daha derin ağlarda “vanishing gradient” sorununun önüne geçmeye yardımcı olduğu için modern ağlarda sıklıkla kullanılır.

input_shape=(10,): Girdi vektörünün boyutunu belirtir.

Daha önce veriyi “5 zaman adımı * 2 değişken = 10 özellik” olacak şekilde flatten ettiğimiz için, burada 10 boyutlu bir giriş beklenir.

input_shape=(10,) demek, “her bir örnek 10 sayıdan oluşuyor” anlamına gelir. İlk katman, bu 10 sayı üzerine kurulur.
----------------------------------------------------------
Neden 64 nöron?
Deneme-yanılma ve deneysel sonuçlara göre seçilebilecek bir hiperparametredir.

Veriye yeteri kadar temsil gücü vermek için 64 genelde “orta ölçekte” bir sayı olabilir. Daha karmaşık veya büyük veri setleri için daha fazla nöron da seçilebilir.
---------------------------------------------------------
3) Dropout(0.2)
Dropout(0.2), bir “düzenlileştirme (regularization)” yöntemidir.

Eğitim sırasında, bu katmanın hemen önceki katmandaki nöronların %20’si (rastgele seçilerek) geçici olarak devre dışı bırakılır (silinir).

Bu sayede ağ, belirli nöronlara veya bağlantılara aşırı bağımlı kalamaz; overfitting (ezberleme) riski azalır.

Tahmin aşamasında (inference) dropout kapatılır, yani tüm nöronlar kullanılır.
------------------------------------------------------
4) Dense(32, activation='relu')
İkinci bir gizli katmandır. 32 nöron içerir, yine ReLU aktivasyon kullanır.

Daha derin bir mimari (birden fazla katman) kullanmak, ağın daha karmaşık ilişkileri öğrenebilmesini sağlar.

Buradaki nöron sayısı yine bir hiperparametredir; 32, 64, 128 gibi farklı değerler denenip en iyi sonuç veren seçilebilir.
------------------------------------------------------
5) Dropout(0.2) (Tekrar)
Bir önceki katmanda da olduğu gibi, yine %20 oranında nöron devre dışı bırakılır.

Modelin derinleşmesiyle birlikte, dropout’un tekrarlanması overfitting’i azaltmada faydalı olabilir.
---------------------------------------------------
6) Dense(2)
Çıkış katmanı: 2 nöron içerir; burada aktivasyon fonksiyonu belirtilmemiş, yani varsayılan olarak “linear” (doğrusal) kullanılır.

Neden 2 nöron? Çünkü aynı anda 2 farklı çıktıyı (SoC ve SoH) tahmin etmek istiyoruz.

Eğer “SoC ve SoH, 0 ile 1 arasında normalize edilmiş” bir regresyon problemi olarak ele alınacaksa, lineer aktivasyon mantıklıdır. Daha sonra scaler.inverse_transform ile orijinal ölçeğe geri dönüyoruz.
-------------------------------------------------

Neden Bu Yapı?
ReLU: Derin ağlarda iyi çalışan, sade ve hızlı bir aktivasyon fonksiyonu.

Dropout: Özellikle kısıtlı veri varsa veya model karmaşıksa overfitting’i düşürmek için sıklıkla kullanılır.

2 Gizli Katman: Tek katmanlı ağın yetersiz kalabileceği durumlarda, ek katman ekleyerek öğrenme kapasitesini artırırız.

2 Çıkış: Tek seferde, 2 ayrı değeri (SoC ve SoH) tahmin edebilmemizi sağlar. Ağ, bu iki hedefi birlikte öğrenir.

In [ ]:
model.compile(optimizer='adam', loss='mse')

Adam algoritması kullanarak optimize eder. loss mse ise mean squared error kullanır

In [ ]:

history = model.fit(X_train, y_train, epochs=50, batch_size=16, validation_data=(X_test, y_test), verbose=1)


MODEL NASIL EĞİTİLİR ?
---------------------------------

Epoch: Tüm eğitim verilerinin (X_train, y_train setinin) bir kez baştan sona ağ tarafından işlenmesidir.

50 epoch demek, verilerinizi 50 kez ardışık şekilde ağdan geçirip, her seferinde ağırlıkları güncelleyeceğiniz anlamına gelir.

Epoch sayısı arttıkça, model veriyi daha çok “görür” ve potansiyel olarak daha iyi öğrenir; ancak aşırı yüksek epoch sayısı, “overfitting” riskini de artırabilir.
-------------------------------------------------------------
batch_size=16

Batch (mini-batch) kavramı, verileri toplu gruplar hâlinde işlemek demektir.

Bir epoch sırasında, veriler tümüyle bir kerede ağırlık güncellemesi yapmak yerine, batch_size kadar örneği modelden geçirip bir “adım” (iteration) yapar, sonra ağırlıkları günceller. Ardından bir sonraki 16 örnekle devam eder.

Neden kullanılır?

Tüm veriyi bir defada geçirmek (batch_size = tüm dataset) hafıza (RAM, GPU VRAM) açısından maliyetli olabilir.

Küçük batch’ler, daha sık güncelleme yaparak ağı daha hızlı veya daha dinamik eğitir.

16 mantıklı bir ara değer: genelde 8, 16, 32, 64 gibi güç-of-2 boyutlar tercih edilir (GPU optimizasyonu vb.).
------------------------------------------------------------
validation_data=(X_test, y_test)

Eğitim sırasında, her epoch bitiminde (veya mini-batch’lerin sonunda) modelin doğruluğunu veya kaybını (loss) test verisi üzerinde ölçer.

Neden?

    Modelin genelleme performansını görür, overfitting olup olmadığını takip edebilirsiniz (örneğin, eğitim kaybı düşerken validasyon kaybı yükseliyorsa aşırı öğrenme başlamıştır).

Bu veri, eğitimde “öğrenilen” veri değildir (genelde train verisinden farklı tutulur). Böylece gerçekte yeni veride modelin nasıl performans göstereceğini yaklaşık tahmin edebilirsiniz.
----------------------------------------------------------------
verbose=1

Eğitim sürecinde, ekrana hangi bilgilerin yazılacağını denetleyen parametredir.

verbose=1 → Her epoch sonu ekrana epoch numarası, loss, val_loss gibi özet bilgileri yazdırır.

verbose=2 → Yine benzer bilgileri yazar ama satırı kısaltır.

verbose=0 → Hiçbir şey yazdırmaz (sessiz mod).
-----------------------------------------------------------------

In [ ]:
predictions = model.predict(X_test)
predictions_rescaled = scaler.inverse_transform(predictions)
y_test_rescaled = scaler.inverse_transform(y_test)

model.predict(X_test)

Eğitilmiş model, test veri seti X_test üzerinde tahmin yürütür.

Modelin son katmanındaki Dense(2)‘nin çıktısı, 2 boyutludur (SoC ve SoH).

Daha önce verilerimiz MinMaxScaler ile 0-1 aralığına (normalleştirilmiş) dönüştürülmüştü; dolayısıyla model de 0-1 aralığında tahmin üretir.

Çıktılar, numpy array formatında (satırlar örneklere, sütunlar 2 farklı hedefe karşılık gelecek şekilde) döner.
------------------------------------------------------------------
predictions_rescaled = scaler.inverse_transform(predictions)

Tahmin sonuçlarını (0-1 aralığında) eski ölçeğe dönüştürmek için kullanılır.

scaler.inverse_transform(...) Metodu, fit_transform sırasında öğrenilen min-max bilgilerini kullanarak, tahminleri orijinal SoC ve SoH aralığına (örneğin 0-100 veya 70-100 aralığı) geri çevirir.

Böylece modelin gerçek hayattaki birimlerle (ör. yüzde % SoC/% SoH) tahmin yapıp yapmadığını görebiliriz.
-----------------------------------------------------------------
Neden Önemli?

Modelin eğitimi boyunca verileri 0-1 aralığına normalleştiririz, bu sayede hesaplamalar (ör. kayıp fonksiyonu, aktivasyon fonksiyonları) genelde daha stabil ve hızlı olur.

Fakat çıktıları yorumlarken çoğu zaman ham değerlere (ör. gerçek SoC ve SoH yüzdeleri) ihtiyaç duyarız. inverse_transform bu dönüşümü sağlar.

Grafikleri veya son kullanıcıya raporlayacağımız çıktıları, bu geri dönüştürülmüş (rescaled) değerlere bakarak sunarız.

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(y_test_rescaled[:, 0], label="Actual SoC", linestyle="dashed")
plt.plot(predictions_rescaled[:, 0], label="Predicted SoC", linestyle="solid")
plt.xlabel("Time Steps")
plt.ylabel("State of Charge (%)")
plt.title("FFNN Prediction of SoC")
plt.legend()
plt.grid(True)
plt.show()

Actual vs Predicted SoC Grafiğini çizer

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(y_test_rescaled[:, 1], label="Actual SoH", linestyle="dashed")
plt.plot(predictions_rescaled[:, 1], label="Predicted SoH", linestyle="solid")
plt.xlabel("Time Steps")
plt.ylabel("State of Health (%)")
plt.title("FFNN Prediction of SoH")
plt.legend()
plt.grid(True)
plt.show()

Actual vs Predicted SoH Grafiğini çizer